# Email Crew


## Load env file


In [1]:
from dotenv import load_dotenv

load_dotenv()


True

## Initialize LLM


In [2]:
from crewai import LLM

llm = LLM(model="gemini/gemini-2.0-flash", temperature=0.1)


## Create a tool


In [3]:
from crewai.tools import BaseTool


class ReplaceJargonsTool(BaseTool):
    name: str = "Replace Jargons"
    description: str = "Replace jargons with specific words"

    def _run(self, email: str) -> str:
        """
        This tool replaces jargons with specific words
        """

        replace_jargons = {
            " :-) ": "smily face",
            "ping me": "reach out to me",
            "wip": "work in progress",
            "eta": "estimated time of arrival",
            "prx": "product requirements exchange",
            "tas": "technical assistance",
            "sds": "software development standards",
            "syncbot": "syncbot",
        }
        suggestions = []
        email_lower = email.lower()
        for jargon, replacement in replace_jargons.items():
            if jargon in email_lower:
                suggestions.append(f"consider replacing {jargon} -> {replacement}")
        return "\n".join(suggestions) if suggestions else "No jargons found"


jt = ReplaceJargonsTool()

## Create a Agent and Task


In [4]:
from crewai import Agent, Task, Crew


def rewrite_email(email_text: str) -> str:
    email_agent = Agent(
        role="Email Agent",
        goal="Send an email to the CEO of the company",
        backstory="You are a helpful assistant that can sends professional emails",
        llm=llm,
        verbose=True,
        tools=[jt],
    )

    email_task = Task(
        description=f"""take the following email and rewrite it to professional version. 
    Original email: '''{email_text}'''
    """,
        agent=email_agent,
        expected_output="An email to the colleagues of the company, expanded the aberrations",
    )

    email_agent_crew = Crew(agents=[email_agent], tasks=[email_task], verbose=True)
    result = email_agent_crew.kickoff()
    return result

## Usage


In [5]:
# Provide your email text and run this cell
email_text = """
hey team, just wanted to tell u that the demo is kind of ready, but there's still stuff left.
Maybe we can show what we have and say rest is WIP.
Let me know what u think. thanks
ping me if any blockers.

PRX and WIP are in the deck.
"""

print(rewrite_email(email_text))


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 2b5175b7-a84c-4075-98bb-8ab96dceb643                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Agent                                                                                             │
│                                                                                                                 │
│  Task: take the following email and rewrite it to professional version.                                         │
│      Original email: '''                                                                                        │
│  hey team, just wanted to tell u that the demo is kind of ready, but there's still stuff left.                  │
│  Maybe we can show what we have and say rest is WIP.                                                            │
│  Let me know what u think. thanks                                                                               │
│  ping me if any blockers.                                                                                       │
│                                                                                                                 │
│  PRX and WIP are in the deck.                                                                                   │
│  '''                                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Agent                                                                                             │
│                                                                                                                 │
│  Thought: I need to rewrite the email to be more professional and expand abbreviations. I can use the Replace   │
│  Jargons tool to achieve this.                                                                                  │
│                                                                                                                 │
│  Using Tool: Replace Jargons                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"email\": \"hey team, just wanted to tell u that the demo is kind of ready, but there's still stuff         │
│  left.\\nMaybe we can show what we have and say rest is WIP.\\nLet me know what u think. thanks\\nping me if    │
│  any blockers.\\n\\nPRX and WIP are in the deck.\"}"                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Agent                                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  "Dear Team,\n\nI wanted to provide an update on the demo. It is nearing completion, although some elements     │
│  are still in progress.\n\nI propose that we present the current version and indicate that the remaining        │
│  features are a work in progress.\n\nPlease share your thoughts on this approach. Thank you.\n\nReach out to    │
│  me if you encounter any obstacles.\n\nProduct Requirements Exchange (PRX) and Work In Progress (WIP) details   │
│  are included in the presentation deck.\n"                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 58f094eb-d59a-4d87-adeb-d2c3145ed6d9                                                                     │
│  Agent: Email Agent                                                                                             │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 2b5175b7-a84c-4075-98bb-8ab96dceb643                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: "Dear Team,\n\nI wanted to provide an update on the demo. It is nearing completion, although     │
│  some elements are still in progress.\n\nI propose that we present the current version and indicate that the    │
│  remaining features are a work in progress.\n\nPlease share your thoughts on this approach. Thank               │
│  you.\n\nReach out to me if you encounter any obstacles.\n\nProduct Requirements Exchange (PRX) and Work In     │
│  Progress (WIP) details are included in the presentation deck.\n"                                               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

"Dear Team,\n\nI wanted to provide an update on the demo. It is nearing completion, although some elements are still in progress.\n\nI propose that we present the current version and indicate that the remaining features are a work in progress.\n\nPlease share your thoughts on this approach. Thank you.\n\nReach out to me if you encounter any obstacles.\n\nProduct Requirements Exchange (PRX) and Work In Progress (WIP) details are included in the presentation deck.\n"
